# How to use Agent

In this notebook we look at simple ways to get started with using Agent.

Note: Please make sure you have first installed all dependencies of Agent, following the installation guide!

## Simple run with start.py

Agent can be run very easily with the default run script `start.py`, which tries to solve the given task, using the given backend LLM and the given method. For exmaple if we want to try to solve questions from the GSM8k task, using OpenChat-3.5 and direct prompting:

! python ../src/agent/start.py task=gsm8k method=direct llm@agent.llm=openchat_3.5 max_episodes=3

## Using different compoennts for your own purposes

You can also the use Agent for your own, customized, purposes. First, initialize the hydra configuration with your desired parameters (method, task, llm, experiment name):

In [ ]:
import hydra
import pyrootutils
from hydra import compose, initialize

from agent.loggers import ManyLoggers, Tag
from agent.utils import pylogger

pyrootutils.setup_root("..", indicator=".project-root", pythonpath=True)
log = pylogger.get_pylogger(__name__)

with initialize(version_base="1.3", config_path="../configs"):
    cfg = compose(
        config_name="default_notebooks.yaml",
        return_hydra_config=True,
        overrides=[
            "method=direct",
            "task=gsm8k",
            "llm@agent.llm=hf/llama-7b",
            "experiment_name=test",
            "logger=stdout_logger",
        ],
    )

Then the different components of Agent can be instantiated:

In [ ]:
logger = ManyLoggers(
    loggers=[
        hydra.utils.instantiate(logger, project_cfg=cfg, _recursive_=False)
        for logger in cfg.logger.values()
    ]
)
agent = hydra.utils.instantiate(cfg.agent, logger=logger)
task = hydra.utils.instantiate(cfg.task)

Start the agent's init flow if present and reset the task and agent, as well as starting the agent's episode start flow:

In [ ]:
agent.run_on_init_flow()

obs = task.reset()
agent.reset(task)

agent.run_on_episode_start_flow()

Now the agent and task can be used as desired.

The agent can observe information about the environment:

In [ ]:
agent.observe(obs)

The agent's pre-action flow can be run (which ends with the agent choosing an external action):

In [ ]:
agent.run_pre_action_flow()

The agent's chosen external action can be executed in the task:

In [ ]:
obs, rew, done = task.step(agent.external_action)

Any post-action flow can be run if present:

In [ ]:
agent.run_post_action_flow()

Multiple rounds of agent observing, pre/post action flows and task steps can be run.

Once an episode ends, the post-episode flow can be run:

In [ ]:
agent.run_on_episode_end_flow()

At any point, metrics can be logged:

In [ ]:
logger.log_metrics({Tag.REWARD: rew, Tag.DONE: done})

And the logs can be saved:

In [ ]:
logger.save_metrics()